In [24]:
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel


In [3]:

filepath = "D:\\MyDramaListScraper\\MyDramaScraper-master\\dramascraper\\dramascraper\\dramas.csv"

df = pd.read_csv(filepath)

display(df.head(3))

,name,synopsis,duration_in_minutes,nb_episodes,country_origin,ratings,ranking,popularity_rank,nb_watchers,nb_ratings,nb_reviews,streamed_on,genres,tags,mydramalist_url,screenwriter,director,main_roles,support_roles,guest_roles
0,Kazoku Game (2013),The story revolves around an eccentric home tu...,46.0,10,Japan,8.6,265,1071,10320,3804,22,NaN,"Mystery,Psychological,Life,Drama","School Bullying,Dysfunctional Family,Hidden Id...",https://mydramalist.com/6068-kazoku-game-2013,Muto Shogo,"Sato Yuichi,Iwata Kazuyuki","Sakurai Sho,Kamiki Ryunosuke,Uragami Seishu,It...","Kutsuna Shiori,Kitahara Rie,Matsushima Kaito,N...","Miyakawa Ichirota,Maeshima Ami,Sakamoto Atsuhi..."
1,HIStory3: Trapped (2019),The story of a police officer who becomes trap...,25.0,20,Taiwan,8.4,440,194,38896,23815,144,"Viki,Netflix,Caravan中文剧场","Action,Thriller,Mystery,Romance","Steamy Kiss,LGBTQ+,Nice Male Lead,Enemies To L...",https://mydramalist.com/30567-history3-trap,"Lin Pei Yu,Shao Hui Ting",Lee Ching Jung,"Jake Hsu,Chris Wu,Andy Bian,Kenny Chen","Diane Lin,Sphinx Ding,Stanley Mei,Chen Chia Ku...","Masuyama Yuki,Jack Wei,Oscar Chiu,Ethan Lo"
2,Voice 2 (2018),"Listen closely, because skilled voice profiler...",60.0,12,South Korea,8.4,444,933,12042,6329,19,"Prime Video,Netflix,K-DRAMA","Thriller,Mystery,Psychological","Investigation,Strong Female Lead,Leadership,Su...",https://mydramalist.com/26153-voice-2,Ma Jin Won,Lee Seung Young,"Lee Ha Na ,Lee Jin Wook","Kwon Yool,Ahn Se Ha,Son Eun Seo,Kim Woo Seok,K...","Kwon Chan Min,Lee Hae Young,So Hee Jung,Jung J..."


In [6]:
# Calculate mean of vote average column
C = df['ratings'].mean()
print(C)

# Calculate the minimum number of votes required to be in the chart, m
m = df['nb_ratings'].quantile(0.65)
print(m)

7.786894757903034
1561.1000000000004


In [7]:
# Filter out all qualified movies into a new DataFrame
q_movies = df.copy().loc[df['nb_ratings'] >= m]
q_movies.shape

(1749, 20)

In [8]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['ratings']
    R = x['nb_ratings']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
df['score'] = df.apply(weighted_rating, axis=1)

In [11]:
#Sort movies based on score calculated above
df = df.sort_values('score', ascending=False)

#Print the top 15 movies
df[['name', 'ratings', 'nb_ratings', 'score']].head(20)

,name,ratings,nb_ratings,score
207,Goblin (2016),8.8,96687,549.717639
358,Strong Woman Do Bong Soon (2017),8.7,96981,545.222845
302,Weightlifting Fairy Kim Bok Joo (2016),8.8,85819,488.797580
348,Descendants of the Sun (2016),8.7,80774,455.401912
226,Crash Landing on You (2019),9.0,77700,453.127904
381,It's Okay to Not Be Okay (2020),9.0,77360,451.178983
327,W (2016),8.5,79070,435.939807
166,What's Wrong with Secretary Kim (2018),8.5,78197,431.212170
243,My Love from the Star (2013),8.6,69930,390.873493
157,Squid Game Season 1 (2021),8.5,68916,380.951912


In [13]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['synopsis'] = df['synopsis'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['synopsis'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4998, 24698)

In [17]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(4998, 4998)

In [18]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [20]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title,dataFrame, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['name'].iloc[movie_indices]

In [23]:
get_recommendations('Vincenzo (2021)',df)

684      She Would Never Know (2021)
2980      The Bride of Habaek (2017)
3922             Jumping Girl (2015)
100                 Angry Mom (2015)
3139        When Time Stopped (2018)
1159       The Chang'an Youth (2020)
4594              Single Papa (2020)
902     Something in the Rain (2018)
291               Racket Boys (2021)
2818                 Lavender (2001)
Name: name, dtype: object

In [26]:
features = ['screenwriter', 'director', 'tags', 'genres', 'main_roles','support_roles']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

SyntaxError: invalid syntax (<unknown>, line 1)